# RAG Retrieve Context

Once the data is uploaded into the vector DB, the next step is connect to an LLM and start retrieving context from the user's query

## Import libraries

In [1]:
from qdrant_client import QdrantClient, models
import sys

sys.path.append("..")

from rag_llm_energy_expert.credentials import get_qdrant_config
from rag_llm_energy_expert.ingest_file_auxiliars import chunk_text, embed_chunks
from utils.vector_db.qdrant import create_points

c:\Users\Emmanuel\Desktop\git_repos\RAG-LLM-Energy-Expert\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
qdrant_config = get_qdrant_config()

In [3]:
qdrant_client = QdrantClient(url = qdrant_config.URL, api_key = qdrant_config.API_KEY.get_secret_value())

In [19]:
def process_query(query:str, embedding_model:str, chunk_overlap):
    # Create chunks based on the query
    chunks=chunk_text(text = query, embedding_model=embedding_model, chunk_overlap=chunk_overlap)

    # Embed the chunks
    chunks_embedded=embed_chunks(chunks=chunks, embedding_model=embedding_model)

    # Obtain the vectors from the chunks
    vectors = [chunk_embed["vector"] for chunk_embed in chunks_embedded]

    return vectors



def process_query_results(results: list):
    full_text = ""

    for query_response in results:

        query_response_points = query_response.points

        for point in query_response_points:

            full_text += point.payload["text"] 

    return full_text  


def semantic_search(
        query:str, 
        embedding_model: str,
        chunk_overlap: int,
        collection_name:str, 
        limit:int
        ):
    """
    
    """
    # Get a list of vector queries
    vectors = process_query(query, embedding_model, chunk_overlap)

    # Create a list of QueryRequest, each QueryRequest object is related to each chunk created from the query
    search_queries = [models.QueryRequest(
    query = vector, 
    with_payload = True,
    with_vector = False,
    limit = limit,
    ) for vector in vectors]


    # Get a list of results from the query batch
    results = qdrant_client.query_batch_points(
        collection_name = collection_name,
        requests= search_queries,

        )
    
    full_text = process_query_results(results)

    return full_text

In [16]:
query = "Cómo se deben desarrollar los contratos para la exploración y extracción de hidrocarburos?"

In [6]:
chunks = chunk_text(text = query, embedding_model=qdrant_config.EMBEDDING_MODEL, chunk_overlap=5)

2025-04-09 05:25:56.186 | INFO     | rag_llm_energy_expert.ingest_file_auxiliars:chunk_text:120 - Chunking text...
!!!!!!!!!!!!megablocks not available, using torch.matmul instead
2025-04-09 05:26:09.183 | INFO     | rag_llm_energy_expert.ingest_file_auxiliars:chunk_text:153 - text successfully chunked


In [7]:
chunks_embedded = embed_chunks(chunks, embedding_model=qdrant_config.EMBEDDING_MODEL)

2025-04-09 05:26:09.432 | INFO     | rag_llm_energy_expert.ingest_file_auxiliars:embed_chunks:180 - Embedding chunks...
2025-04-09 05:26:24.970 | INFO     | rag_llm_energy_expert.ingest_file_auxiliars:embed_chunks:221 - Chunks successfully embedded


In [8]:
len(chunks_embedded[0]["vector"])

768

In [9]:
len(chunks_embedded)

1

In [10]:
vectors = [chunk_embed["vector"] for chunk_embed in chunks_embedded]

In [11]:
search_queries = [models.QueryRequest(
    query = vector, 
    with_payload = True,
    with_vector = False,
    limit = 20) for vector in vectors]

search_queries

[QueryRequest(shard_key=None, prefetch=None, query=[0.037333421409130096, 0.009628607891499996, -0.005247393622994423, 0.06747587770223618, -0.003568746382370591, -0.0040047913789749146, 0.05176210403442383, -0.04963967949151993, -0.05247596651315689, 0.0527954138815403, -0.006986875552684069, 0.0016101084183901548, -0.01935749687254429, 0.06196287274360657, -0.001375086372718215, 0.03511467203497887, 0.06555018573999405, -0.059915691614151, -0.04290297254920006, 0.020918289199471474, -0.015295393764972687, 0.03133336454629898, 0.02572369948029518, -0.038177378475666046, -0.012782994657754898, -0.04147658124566078, 0.0377291776239872, -0.026325082406401634, 0.05375779792666435, 0.006127423606812954, -0.005301188677549362, -0.06336180865764618, -0.012525014579296112, 0.008161226287484169, 0.03554338589310646, 0.03155722841620445, 0.029992401599884033, -0.05964607745409012, 0.07783853262662888, 0.022204099223017693, 0.013140635564923286, -0.014614978805184364, 0.0479920357465744, 0.02840

In [12]:
collection_name = qdrant_config.COLLECTION_NAME + qdrant_config.COLLECTION_VERSION

results = qdrant_client.query_batch_points(
    collection_name = collection_name,
    requests= search_queries,
)

In [14]:
full_text = ""

for query_response in results:

    query_response_points = query_response.points

    for point in query_response_points:

        full_text += point.payload["text"] 

In [15]:
print(full_text)

 las centrales eléctricas más contaminantes con tecnologías limpias y fomentar la 
utilización de gas natural en la generación eléctrica.

4
II. Abasto de energéticos a precios competitivos
i) 	 Nuevo modelo de producción de petróleo y gas natural
México, al igual que muchos otros países con abundantes recursos naturales, sustenta en gran 
medida su desarrollo económico y social en la riqueza energética. Sin embargo, el sector petro-
lero mexicano enfrenta retos considerables y urgentes por atender, entre los que se encuentran 
la caída de la producción y la necesidad de mayor inversión a nivel industria.
A pesar de invertir más en exploración y extracción de petróleo y gas natural, la producción de 
petróleo pasó de 3.4 millones de barriles diarios en 2004, a 2.5 millones de barriles diarios en 
2013. Con respecto al gas natural, en 1997 México era prácticamente autosuficiente, pues 
sólo importábamos 3% del consumo nacional. En la actualidad importamos 30% del gas natu-
ral que consu

In [23]:
context = semantic_search(query, embedding_model=qdrant_config.EMBEDDING_MODEL, chunk_overlap=qdrant_config.CHUNK_OVERLAP, collection_name=collection_name, limit = 5)

print(context)

2025-04-09 05:35:53.512 | INFO     | rag_llm_energy_expert.ingest_file_auxiliars:chunk_text:120 - Chunking text...
2025-04-09 05:36:06.658 | INFO     | rag_llm_energy_expert.ingest_file_auxiliars:chunk_text:153 - text successfully chunked
2025-04-09 05:36:06.857 | INFO     | rag_llm_energy_expert.ingest_file_auxiliars:embed_chunks:180 - Embedding chunks...
2025-04-09 05:36:16.637 | INFO     | rag_llm_energy_expert.ingest_file_auxiliars:embed_chunks:221 - Chunks successfully embedded


ón de gas natural y petróleo, de sus-
cribirlos y administrarlos de manera técnica.
Legislación Secundaria
• Para la realización de las actividades de exploración y extracción de hidrocarburos el Estado 
tiene la posibilidad de otorgar asignaciones o suscribir contratos.

6
Asignaciones 
• Se otorgarán a Pemex en la Ronda Cero.
• Posteriormente, se otorgan de forma excepcional a Pemex y a otras 
Empresas Productivas del Estado.
• Las asignaciones permiten la adjudicación directa a Pemex de pro-
yectos estratégicos tales como yacimientos transfronterizos.
• Los asignatarios podrán registrar el beneficio económico esperado 
para efectos financieros y contables.
Contratos para la Exploración y Extracción
 
• Los hidrocarburos en el subsuelo son propiedad de la Nación.
• Los contratos pueden ser suscritos con a) Pemex, b) Pemex asocia-
do con particulares o c) particulares.
• Adjudicación por licitación a quien ofrezca las mejores condiciones 
económicas para el Estado y el mayor compromis